# Linear Regression

Linear methods assume that there is a linear relationship between the features and the target/label, that is
$$y = w_1 x_1 + w_2 x_2 + w_x x_k + b$$
where $y$ - target (what we want to predict), $x_i$ - feature of $x$, $w_i$ - weight of $i$-th feature, $b$ - bias.

## Formula
$x_i$ and $y_i$ is $x$ and $y$ coordinate of $i$-th point
$n$ is length of input
$$
intercept = \frac{ \sum{x_i^2} \cdot \sum{y_i} - \sum{x_i} \cdot \sum{x_i y_i}  }{n \sum{x_i^2} - (\sum{x_i})^2}
$$

$$
slope = \frac{n \sum{x_i y_i} - \sum{x_i} \cdot  \sum{y_i}}{n \sum{x_i^2} - (\sum{x_i})^2}
$$

In [3]:
import numpy as np


class LinearRegression:
    def __init__(self, fit_intercept=True):
        self.fitted = False
        self.fit_intercept = fit_intercept
        self.w = None

    def fit(self, X, y):
        n, k = X.shape

        X_train = X
        if self.fit_intercept:
            X_train = np.hstack((X, np.ones((n, 1))))

        self.w = np.linalg.inv(X_train.T @ X_train) @ X_train.T @ y

        self.fitted = True
        return self

    def fitted(self):
        return self.fitted

    def predict(self, X):
        if not self.fitted:
            return "Not fitted"
        n, k = X.shape
        if self.fit_intercept:
            X_train = np.hstack((X, np.ones((n, 1))))

        y_pred = X_train @ self.w

        return y_pred

    def get_weights(self):
        return self.w

In [4]:
from sklearn.metrics import mean_squared_error


class MyGradientLinearRegression(LinearRegression):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.w = None

    def fit(self, X, y, lr=0.01, max_iter=100):
        n, k = X.shape

        if self.w is None:
            self.w = np.random.randn(k + 1 if self.fit_intercept else k)

        X_train = np.hstack((X, np.ones((n, 1)))) if self.fit_intercept else X

        self.losses = []

        for iter_num in range(max_iter):
            y_pred = self.predict(X)
            self.losses.append(mean_squared_error(y_pred, y))

            grad = self._calc_gradient(X_train, y, y_pred)

            assert grad.shape == self.w.shape, f"gradient shape {grad.shape} is not equal weight shape {self.w.shape}"
            self.w -= lr * grad

        return self

    def _calc_gradient(self, X, y, y_pred):
        grad = 2 * (y_pred - y)[:, np.newaxis] * X
        grad = grad.mean(axis=0)
        return grad

    def get_losses(self):
        return self.losses

In [ ]:
class MySGDLinearRegression(MyGradientLinearRegression):
    def __init__(self, n_sample=10, **kwargs):
        super().__init__(**kwargs)
        self.w = None
        self.n_sample = n_sample

    def _calc_gradient(self, X, y, y_pred):
        inds = np.random.choice(np.arange(X.shape[0]), size=self.n_sample, replace=False)

        grad = 2 * (y_pred[inds] - y[inds])[:, np.newaxis] * X[inds]
        grad = grad.mean(axis=0)

        return grad